## Banning Transit is missing
[PR](https://github.com/cal-itp/data-analyses/issues/1254)

In [6]:
import geopandas as gpd
import merge_data
import numpy as np
import pandas as pd
from segment_speed_utils import gtfs_schedule_wrangling, helpers, time_series_utils
from shared_utils import catalog_utils, rt_dates, rt_utils
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS

In [7]:
from segment_speed_utils.project_vars import (
    COMPILED_CACHED_VIEWS,
    GTFS_DATA_DICT,
    PROJECT_CRS,
    RT_SCHED_GCS,
    SCHED_GCS,
    SEGMENT_GCS,
)

In [8]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [9]:
import _operators_prep as op_prep

In [10]:
analysis_date_list = rt_dates.y2024_dates + rt_dates.y2023_dates

## 11/18: why isn't `deploy_portfolio_yaml` working

In [11]:
df = (
    op_prep.operators_schd_vp_rt()
    .sort_values(["caltrans_district", "organization_name"])
    .reset_index(drop=True)
)

In [13]:
df.head()

,caltrans_district,organization_name
0,01 - Eureka,Blue Lake Rancheria
1,01 - Eureka,City of Arcata
2,01 - Eureka,City of Eureka
3,01 - Eureka,Curry Public Transit
4,01 - Eureka,Humboldt Transit Authority


In [14]:
df.caltrans_district.value_counts()

07 - Los Angeles        54
04 - Oakland            34
03 - Marysville         18
10 - Stockton           16
05 - San Luis Obispo    12
06 - Fresno             12
08 - San Bernardino     11
01 - Eureka              9
02 - Redding             9
11 - San Diego           6
12 - Irvine              3
09 - Bishop              1
Name: caltrans_district, dtype: int64

In [1]:
chapter_info = {
    "column": "caltrans_district",
    "name": "district",
    "caption_prefix": "District ",
    "caption_suffix": "",
}

In [2]:
section_info = {
    "column": "organization_name",
    "name": "organization_name",
}

In [3]:
chapter_col = chapter_info["column"]

In [4]:
chapter_col

'caltrans_district'

In [5]:
chapter_values = sorted(list(df[chapter_col].unique()))

NameError: name 'df' is not defined

In [ ]:
import yaml
from pathlib import Path

In [ ]:
PORTFOLIO_SITE_YAML = Path("../portfolio/sites/gtfs_digest.yml")

In [ ]:
prefix = chapter_info["caption_prefix"]

In [ ]:
section_col = section_info["column"]

In [ ]:
with open(PORTFOLIO_SITE_YAML) as f:
        site_yaml_dict = yaml.load(f, yaml.Loader)

In [ ]:
chapters_list = []
for district in chapter_values:
    df2 = df.loc[df.caltrans_district == district]
    chapters_list_one_district = [
    {
        **{
            "caption": f"{prefix} {district}",
            "params": {chapter_info["name"]: district},
            "section": [
                {section_info["name"]: one_section_value}
                for one_section_value in df2[section_col].unique().tolist()
            ],
        }
    }]
    chapters_list.append(chapters_list_one_district)

In [ ]:
chapters_list[0]

In [ ]:
parts_list = [{"chapters": chapters_list}]

In [ ]:
type(parts_list)

In [ ]:
site_yaml_dict["parts"] = parts_list

In [ ]:
site_yaml_dict

In [ ]:
output = yaml.dump(site_yaml_dict)

In [ ]:
with open(PORTFOLIO_SITE_YAML, "w") as f:
        f.write(output)

## 11/18: double check the yaml generation

In [ ]:
stop

In [ ]:
schd_vp_url = f"{GTFS_DATA_DICT.digest_tables.dir}{GTFS_DATA_DICT.digest_tables.route_schedule_vp}.parquet"

In [ ]:
schd_vp_df = pd.read_parquet(
    schd_vp_url,
    filters=[[("sched_rt_category", "in", ["schedule_and_vp", "schedule_only"])]],
    columns=[
        "schedule_gtfs_dataset_key",
        "caltrans_district",
        "organization_name",
        "name",
        "sched_rt_category",
        "service_date",
    ],
)

In [ ]:
schd_vp_df.shape

In [ ]:
schd_vp_df.sched_rt_category.value_counts()

In [ ]:
schd_vp_df.info()

In [ ]:
schd_vp_df.loc[schd_vp_df.caltrans_district.isna()][
    ["organization_name"]
].drop_duplicates()

In [ ]:
schd_vp_df2 = (
    schd_vp_df.dropna(subset="caltrans_district")
    .sort_values(
        by=[
            "caltrans_district",
            "organization_name",
            "service_date",
        ],
        ascending=[True, True, False],
    )
    .drop_duplicates(
        subset=[
            "organization_name",
            "caltrans_district",
        ]
    )
    .reset_index(drop=True)
)

In [ ]:
schd_vp_df2.shape

In [ ]:
schd_vp_df2.sched_rt_category.value_counts()

In [ ]:
schd_vp_df2.organization_name.nunique()

In [ ]:
schd_vp_df2.service_date.value_counts()

In [ ]:
# schd_vp_df2

In [ ]:
df = (
    op_prep.operators_schd_vp_rt()
    .sort_values(["caltrans_district", "organization_name"])
    .reset_index(drop=True)
)

In [ ]:
df